# About
* **Author**: Adil Rashitov
* **Created at**: 05.08.2021
* **Goal**: Solve final assignment

![pictures](pictures/FINAL_TASK_1.png)
![Formulation](pictures/OR_2_FINAL.drawio.png)

In [4]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from ortools.linear_solver import pywraplp

'Linux'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [5]:
# Data reading & prepatre
def prepare_data():

    DATA = """
    1	7	None
    2	4	5, 8
    3	6	None
    4	9	None
    5	12	2, 8
    6	8	9
    7	10	10
    8	11	2, 5
    9	8	6
    10	7	7
    11	6	15
    12	8	None
    13	15	None
    14	14	None
    15	3	11
    """


    DATA = DATA.split('\n')[1:-1]
    DATA = list(map(lambda x: x.split('\t'), DATA))
    data = pd.DataFrame(DATA, columns=['Job', 'Processing time', 'Conflicting jobs'])
    return data

df = prepare_data()

In [6]:
N_MACHINES = 3

# Main


## 1. Inputs prepare

In [74]:
jobs_vec = list(map(int, df['Processing time'].values))
jobs_vec

[7, 4, 6, 9, 12, 8, 10, 11, 8, 7, 6, 8, 15, 14, 3]

In [77]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

# 1. Vector of time required to process specific job
vec_job_proc_time = np.array(df['Processing time'].astype(int))
N_JOBS = vec_job_proc_time.shape[0]

# 2. X matrix definition (machine -> jobs assignment matrix of booleans)
X = [ [model.NewBoolVar(f"x_{i}_{j}") for i in range(N_JOBS)] for j in range(N_MACHINES)] 

## 2. Objective funciton

Approaches:
1. Assignment of constrain output `AddMaxEquality` to `obj_value` -> optimization `obj_value`
2. Direct specification of objective function

In [79]:
obj_value = model.NewIntVar(0, int(vec_job_proc_time.sum()), 'makespan')
model.AddMaxEquality(obj_value, X @ jobs_vec)

TypeError: unsupported operand type(s) for @: 'list' and 'list'

In [61]:
# 2. Approach set 2: Direct specification of objective functions
# model.Minimize(max(d_matrix @ vec_job_proc_time))
max(d_matrix @ vec_job_proc_time)



NotImplementedError: Evaluating a BoundedLinearExpr as a Boolean value is not supported.

In [99]:
np.max(d_matrix @ vec_job_proc_time)

NotImplementedError: Evaluating a BoundedLinearExpr as a Boolean value is not supported.

In [98]:
from copy import deepcopy


def matrix_mul(A, B):

    A = np.array(A)
    B = np.array(B)

    new_matrix = []

    for i, row in enumerate(A):
        new_row = []
        for j, col in enumerate(B.T):
            dot_product = sum([x*y for (x, y) in zip(row, col)])
            new_row.append(dot_product)
        
        new_matrix.append(new_row)
    
    return new_matrix


max(list(map(sum, matrix_mul(deepcopy(X), deepcopy([jobs_vec])))))

NotImplementedError: Evaluating a BoundedLinearExpr as a Boolean value is not supported.

## 3. Constrains
1. **Conflicting jobs constrains**: Jobs can't be processed on the same machine
2. **Unique machine assignment**: Each job assigned at most one worker

### 1. Conflicting jobs constrains

In [23]:
# Jobs data
# 1. Extraction of conflicting jobs
constrains = df.loc[df['Conflicting jobs'] != 'None', ['Job', 'Conflicting jobs']].copy(deep=True)
constrains['Conflicting jobs'] = constrains['Conflicting jobs'].str.split(', ')
constrains = constrains.explode('Conflicting jobs')
constrains = constrains.replace('  ', '', regex=True)

# 2. Extraction of only unique constrain pairs
unique_conflicting_job_pairs = set(list(zip(constrains.T.min(), constrains.T.max())))
constrains = pd.DataFrame(unique_conflicting_job_pairs).astype(int).sort_values([0, 1])



### 2. Unique machine assignment

In [24]:
sum_n_machines_running_job_at_time = list(map(np.sum, np.transpose(d_matrix)))


for n_machines_running_job_at_time in sum_n_machines_running_job_at_time:
    model.Add(n_machines_running_job_at_time <= 1)